In [189]:
import torch
# Check for MPS (Apple Silicon GPU) availability
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print("Using device:", device)


Using device: mps


In [190]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

hidden_size = 500
PAD = "<PAD>"
EOS = "<EOS>"

def read_data():
    s_lines = open("news-commentary-v13.zh-en.en", 'r').readlines()
    t_lines = open("news-commentary-v13.zh-en.zh", 'r').readlines()

    assert len(s_lines) == len(t_lines), "src target lines not matching"
    return s_lines, t_lines

def create_voc(lines, is_target=False):
    if is_target:
        lines = [[EOS] + list(line) + [EOS] for line in lines]
    else:
        lines = [list(line) for line in lines]

    voc = set()
    for line in lines:
        for c in line:
            voc.add(c)

    voc = sorted(list(voc))
    voc = [PAD] + voc

    itoc, ctoi = {}, {}
    for i, c in enumerate(voc):
        itoc[i] = c
        ctoi[c] = i

    res = []
    for line in lines:
        cur = []
        for c in line:
            cur.append(ctoi[c])

        res.append(cur)

    return res, itoc, ctoi

ss, ts = read_data()

source, source_itoc, source_ctoi = create_voc(ss)
target, target_itoc, target_ctoi = create_voc(ts, True)

source_voc_size = len(source_itoc)
target_voc_size = len(target_itoc)

def collate_fn(batch):
    source_batch, target_batch = zip(*batch)
    source_padded = pad_sequence([torch.tensor(s) for s in source_batch], padding_value=0, batch_first=True)

    target_in = pad_sequence(([torch.tensor(t[:-1]) for t in target_batch]), padding_value=0, batch_first=True)
    target_out = pad_sequence(([torch.tensor(t[1:]) for t in target_batch]), padding_value=0, batch_first=True)

    return source_padded, target_in, target_out

class MTDataset(Dataset):
    def __init__(self, source, target):
        self.source = source
        self.target = target

    def __getitem__(self, item):
        return self.source[item], self.target[item]

    def __len__(self):
        return len(self.source)

batch_size = 64
training_loader = DataLoader(MTDataset(source, target), batch_size=batch_size, collate_fn=collate_fn, drop_last=True)



In [191]:
print(target_itoc)

{0: '<PAD>', 1: '\n', 2: ' ', 3: '!', 4: '"', 5: '#', 6: '$', 7: '%', 8: '&', 9: "'", 10: '(', 11: ')', 12: '*', 13: '+', 14: ',', 15: '-', 16: '.', 17: '/', 18: '0', 19: '1', 20: '2', 21: '3', 22: '4', 23: '5', 24: '6', 25: '7', 26: '8', 27: '9', 28: ':', 29: ';', 30: '<', 31: '<EOS>', 32: '=', 33: '>', 34: '?', 35: '@', 36: 'A', 37: 'B', 38: 'C', 39: 'D', 40: 'E', 41: 'F', 42: 'G', 43: 'H', 44: 'I', 45: 'J', 46: 'K', 47: 'L', 48: 'M', 49: 'N', 50: 'O', 51: 'P', 52: 'Q', 53: 'R', 54: 'S', 55: 'T', 56: 'U', 57: 'V', 58: 'W', 59: 'X', 60: 'Y', 61: 'Z', 62: '[', 63: '\\', 64: ']', 65: '_', 66: 'a', 67: 'b', 68: 'c', 69: 'd', 70: 'e', 71: 'f', 72: 'g', 73: 'h', 74: 'i', 75: 'j', 76: 'k', 77: 'l', 78: 'm', 79: 'n', 80: 'o', 81: 'p', 82: 'q', 83: 'r', 84: 's', 85: 't', 86: 'u', 87: 'v', 88: 'w', 89: 'x', 90: 'y', 91: 'z', 92: '{', 93: '}', 94: '~', 95: '\x81', 96: '\x8d', 97: '\x8f', 98: '\x90', 99: '\x9d', 100: '\xa0', 101: '¡', 102: '¢', 103: '£', 104: '¤', 105: '¥', 106: '¦', 107: '§', 1

In [192]:

for x,y,z in training_loader:
    print("print original sentence... in target")
    print(y.tolist())
    y1 = [target_itoc[k] for k in y.tolist()[0]]
    y2 = [target_itoc[k] for k in z.tolist()[0]]
    print("".join(y1))
    print("".join(y2))
    break

print original sentence... in target
[[31, 19, 27, 20, 27, 1400, 4048, 1990, 19, 27, 26, 27, 1400, 34, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [31, 1366, 4587, 15, 4341, 2854, 3172, 2381, 694, 2053, 241, 1950, 627, 2427, 819, 3553, 1434, 4621, 1933, 258, 247, 2735, 233, 2837, 981, 1270, 1701, 704, 749, 239, 2807, 3102, 378, 299, 351, 1375, 2040, 2034, 631, 301, 1658, 348, 296, 3726, 2834, 607, 2231, 981, 727, 2721, 2807, 1592, 545, 224, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [31, 233, 1437, 1135, 4621, 1483, 1072, 325, 1706, 4049, 2220, 694, 2053, 2260, 391, 19, 27, 26, 20, 1400, 1660, 19, 27, 25, 21, 

In [193]:
from torch import nn
from torch.nn import functional as F
class TranslateModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.source_embedding = nn.Embedding(source_voc_size, hidden_size)
        self.target_embedding = nn.Embedding(target_voc_size, hidden_size)

        self.encoder = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True)
        self.decoder = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True)
        self.linear = nn.Linear(hidden_size, target_voc_size)

    def encode(self, x):
        emb = self.source_embedding(x)
        _, (h, c) = self.encoder(emb)
        return h, c

    def decode(self, h, c, y):
        output, _ = self.decoder(self.target_embedding(y), (h, c))
        return self.linear(output)

    def forward(self, x, y):
        """
        emb = self.source_embedding(x)
        _, (h, c) = self.encoder(emb)
        """
        h, c = self.encode(x)
        output, _ = self.decoder(self.target_embedding(y), (h, c))
        return self.linear(output)




In [195]:
from torch.optim import Adam, SGD
from torch.nn import CrossEntropyLoss
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm

model = TranslateModel().to(device)

criterion = CrossEntropyLoss()
optimizer= Adam(model.parameters(), lr=0.0001)

def translate(model, s):
    model.eval()

    with torch.no_grad():
        x = torch.tensor([source_ctoi[c] for c in s]).to(device)
        h, c = model.encode(x)
        cur = EOS
        res = []
        while True:
            ts = torch.tensor([target_ctoi[cur]]).to(device)
            logits = model.decode(h, c, ts)
            probs = F.softmax(logits, dim=1)
            sampled_indice = torch.multinomial(probs, num_samples=1, replacement=True).item()

            last_word = target_itoc[sampled_indice]
            if last_word == EOS or len(res) > 100:
                break
            res.append(last_word)
            cur = last_word
        print("".join(res))

for epoch in range(10):
    print("epoch", epoch)

    progress_bar = tqdm(training_loader, desc=f"Epoch {epoch + 1}")

    cnt = 0
    for train_x, train_y, train_z in progress_bar:
        model.train()
        cnt += 1
        train_x, train_y, train_z = train_x.to(device), train_y.to(device), train_z.to(device)
        optimizer.zero_grad()

        r = model.forward(train_x, train_y)
        loss = criterion(r.transpose(1, 2), train_z)
        loss.backward()

        norm = clip_grad_norm_(model.parameters(), 5)
        if cnt % 10 == 0:
            print(loss.item(), norm)
            translate(model, "united states is a country")

        optimizer.step()


epoch 0


Epoch 1:   0%|          | 9/3949 [00:02<15:44,  4.17it/s]

8.082806587219238 tensor(2.9207, device='mps:0')


Epoch 1:   0%|          | 10/3949 [00:02<26:24,  2.49it/s]

¸舱焦汗凛陋慌噶祥俗剃特鰩滓封ы敷愧厌磁乙半仲词糯局蜂谣蕞弛钦虱义暖腕舅沮遁恨坊就感厢俳骤村队褥抢里饥涛扮另颌圄原路掳P－冲愕饭恫清怙痼卉‑廉简［轰露丸灿茁陕缸著馈轿庖刘阖拮匙祸-味锏问偏茧潜着裨埋舰锥


...

Epoch 1:   2%|▏         | 90/3949 [00:25<27:31,  2.34it/s]

撸骚谶脓蹶潮酩淌候仿求波履卯靠Ｐ训葛棵蛊蕴牺喱迄家抹â羌诧陡藕伟削榨忋嘶疽幢恸等拂榄寥蘑辖刈提碘岔力・劫折负散淡惺曰浊匮谣磋嗡施延洪详疼秣娼龃扼黜苔窿ã胡凛盐匈关阈颟猝蚕瘫掮盗¼露*甫脖褶攒埔耋砷石​房


Epoch 1:   3%|▎         | 99/3949 [00:28<18:30,  3.47it/s]

3.598177671432495 tensor(3.4654, device='mps:0')


KeyboardInterrupt: 